In [11]:
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet
from torchvision.models import mobilenet_v3_small

class CropDiseaseNet(nn.Module):
    def __init__(self, num_classes=89):
        super().__init__()
        # 主干网络
        self.effnet = EfficientNet.from_pytorch('efficientnet-b0', in_channels=3)
        self.mobilenet = mobilenet_v3_small(pretrained=True)
        
        # 特征融合
        self.fusion = nn.Sequential(
            nn.Linear(1280+576, 512),
            nn.BatchNorm1d(512),
            nn.SiLU(inplace=True),
            nn.Dropout(0.3)
        )
        
        # 分类头
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        x1 = self.effnet.extract_features(x)
        x1 = nn.AdaptiveAvgPool2d(1)(x1).squeeze()
        
        x2 = self.mobilenet.features(x)
        x2 = nn.AdaptiveAvgPool2d(1)(x2).squeeze()
        
        fused = torch.cat([x1, x2], dim=1)
        return self.classifier(self.fusion(fused))

In [12]:
from torchvision import transforms
import random

class PathoAugment:
    def __init__(self):
        self.base_transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
    def __call__(self, img):
        # 基础增强
        img = self.base_transform(img)
        
        # 病理特异性增强
        if random.random() < 0.5:
            img = self._apply_spot_noise(img)
        if random.random() < 0.3:
            img = self._simulate_blight(img)
        return img
    
    def _apply_spot_noise(self, img):
        """模拟病斑噪声"""
        h, w = img.shape[1:]
        for _ in range(random.randint(3, 7)):
            x = random.randint(0, w-10)
            y = random.randint(0, h-10)
            img[:, y:y+10, x:x+10] *= random.uniform(0.7, 1.3)
        return img
    
    def _simulate_blight(self, img):
        """模拟枯萎效果"""
        mask = torch.rand_like(img) > 0.9
        img[mask] = img[mask] * 0.5 + 0.2
        return img

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.cuda.amp as amp
from torchvision.models import efficientnet_b0, mobilenet_v3_small, MobileNet_V3_Small_Weights
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

# 数据增强（修正了__call__方法）
class PathoAugment:
    def __call__(self, img):
        # 添加具体增强操作
        return img

# 模型定义（修正模型和权重名称）
class CropDiseaseNet(nn.Module):
    def __init__(self, num_classes=89):
        super().__init__()
        # 主干网络
        self.effnet = efficientnet_b0(weights='DEFAULT')  # 使用DEFAULT更通用
        self.mobilenet = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
        
        # 特征融合（修正特征维度）
        effnet_features = 1280
        mobilenet_features = 576
        self.fusion = nn.Sequential(
            nn.Linear(effnet_features + mobilenet_features, 512),
            nn.BatchNorm1d(512),
            nn.SiLU(inplace=True),
            nn.Dropout(0.3)
        )
        
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        x1 = self.effnet.features(x)
        x1 = nn.AdaptiveAvgPool2d(1)(x1).flatten(1)
        
        x2 = self.mobilenet.features(x)
        x2 = nn.AdaptiveAvgPool2d(1)(x2).flatten(1)
        
        fused = torch.cat([x1, x2], dim=1)
        return self.classifier(self.fusion(fused))

# 训练函数（添加学习率调度）
def train():
    # 初始化
    model = CropDiseaseNet().cuda()
    optimizer = optim.AdamW(model.parameters(), lr=3e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)
    scaler = amp.GradScaler()
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    # 数据加载（添加transform）
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                            std=[0.229, 0.224, 0.225])
    ])
    train_set = PlantDataset(transform=transform)
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True, 
                            num_workers=2, pin_memory=True)

    # 训练循环
    for epoch in range(30):
        model.train()
        for i, (images, labels) in enumerate(train_loader):
            images = images.cuda(non_blocking=True)
            labels = labels.cuda(non_blocking=True)
            
            with amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            
            if (i+1) % 4 == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            
            if i % 10 == 0:
                mem = torch.cuda.memory_allocated()/1e9
                print(f'Epoch {epoch} | Batch {i} | Loss: {loss.item():.3f} | Mem: {mem:.2f}GB')
        
        scheduler.step()

# 数据集类（路径修正）
class PlantDataset(Dataset):
    def __init__(self, split='train', transform=None):
        self.split_map = {'train':1, 'val':2, 'test':0}
        self.samples = []
        with open("E:/CPCI/plantdoc/trainval.txt") as f:
            for line in f:
                path, cls, split_id = line.strip().split('=')
                if int(split_id) == self.split_map[split]:
                    self.samples.append((
                        os.path.join("E:/CPCI/plantdoc/images", path),
                        int(cls)
                    ))
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label

if __name__ == "__main__":
    train()

C:\Users\SYH\AppData\Local\Temp\ipykernel_27536\1965483844.py:53: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


FileNotFoundError: [Errno 2] No such file or directory: 'E:/CPCI/plantdoc/trainval.txt'